In [9]:
using BenchmarkTools
using Random

# Define a simple UCB function
function ucb(mu::Vector, σ²::Vector; kappa::Real=2.0)
    return mu .+ kappa .* sqrt.(σ²)
end

# Example usage
Random.seed!(123)
mu = rand(10)
σ² = rand(10)

@benchmark ucb_values = ucb(mu, σ²)


BenchmarkTools.Trial: 10000 samples with 800 evaluations.
 Range (min … max):  149.500 ns …   5.444 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     164.125 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   192.902 ns ± 192.419 ns  ┊ GC (mean ± σ):  1.69% ± 4.08%

  ▂█▄▃▁▁▁▁▁    ▁                                                ▁
  ████████████████▇▇▇▇▇▅▆▆▆▅▆▆▅▅▄▅▅▁▅▄▄▅▅▄▄▅▁▅▅▃▅▁▃▄▃▁▃▄▁▄▄▄▄▁▄ █
  150 ns        Histogram: log(frequency) by time        731 ns <

 Memory estimate: 144 bytes, allocs estimate: 1.

In [3]:
using Random

# Define the Ackley function with added noise
function ackley(x::Vector{T}, noise_level::Float64) where T
    n = length(x)
    sum1 = sum(x.^2)
    sum2 = sum(cos.(2π .* x))
    noise = noise_level * randn()
    return -20exp(-0.2sqrt(sum1/n)) - exp(sum2/n) + 20 + ℯ + noise
end

ackley (generic function with 1 method)

## UCB Optimization

In [4]:
using Random

# Upper Confidence Bound (UCB) optimization algorithm
function ucb_optimization(obj_func, dim::Int; num_iterations::Int=100)
    X = randn(num_iterations, dim)  # Initialize points randomly
    Y = [obj_func(X[i, :]) for i in 1:num_iterations]  # Evaluate objective function
    best_point = argmin(Y)  # Initialize best point
    for i in num_iterations+1:num_iterations+1000
        next_point = argmax(Y .+ 2sqrt.(log(i)/length(Y)))  # UCB formula
        X_new = randn(dim)  # Generate new random point
        Y_new = obj_func(X_new)  # Evaluate objective function at new point
        if Y_new < Y[next_point]
            X[next_point, :] = X_new  # Replace point with lower objective value
            Y[next_point] = Y_new
            if Y_new < Y[best_point]
                best_point = next_point  # Update best point if better solution found
            end
        end
    end
    return X[best_point, :], Y[best_point]
end

# Test UCB optimization on Ackley function
Random.seed!(123)
best_point, best_value = ucb_optimization(ackley, 2)

# Store the best point and best value in variables
best_point_found = best_point
best_value_found = best_value

ucb_best_point = best_point
ucb_best_value = best_value

println("Best point found: ", best_point_found)
println("Best value found: ", best_value_found)


MethodError: MethodError: no method matching ackley(::Vector{Float64})

Closest candidates are:
  ackley(::Vector{T}, !Matched::Float64) where T
   @ Main c:\Users\mridu\Desktop\Julia\temp.ipynb:4


## RTEA Optimization

In [15]:
using Random

# RTEA optimization algorithm
function rtea_optimization(obj_func, dim::Int; num_iterations::Int=100)
    # Initialize population with random points
    population = randn(num_iterations, dim)
    best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
    
    # Iterate and evolve the population
    for iter in 1:1000
        # Randomly select two points from the population
        idx1, idx2 = rand(1:num_iterations, 2)
        p1, p2 = population[idx1, :], population[idx2, :]
        
        # Crossover: Create a new point by averaging the selected points
        new_point = (p1 + p2) / 2
        
        # Mutation: Perturb the new point with random noise
        mutation_scale = 0.1
        new_point += mutation_scale * randn(dim)
        
        # Evaluate the objective function at the new point
        new_value = obj_func(new_point)
        
        # Replace the worst point in the population with the new point if it's better
        if new_value < obj_func(population[best_point, :])
            population[best_point, :] = new_point
            best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
        end
    end
    
    # Return the best point found
    return population[best_point, :]
end

# Test RTEA optimization on the Ackley function
Random.seed!(123)
best_point_found = rtea_optimization(ackley, 2)
best_value_found = ackley(best_point_found)

rtea_best_point = best_point_found
rtea_best_value = best_value_found

println("Best point found: ", best_point_found)
println("Best value found: ", best_value_found)


Best point found: [-0.001226936968122122, 0.005485390097988544]
Best value found: 0.016739506567951867


## EI Optimiztion

In [17]:
using Random

# EI optimization algorithm
function ei_optimization(obj_func, dim::Int; num_iterations::Int=100)
    # Initialize population with random points
    population = randn(num_iterations, dim)
    best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
    
    # Iterate and improve the population
    for iter in 1:1000
        # Randomly select a point from the population
        idx = rand(1:num_iterations)
        p = population[idx, :]
        
        # Perturb the selected point with random noise
        mutation_scale = 0.1
        new_point = p + mutation_scale * randn(dim)
        
        # Evaluate the objective function at the new point
        new_value = obj_func(new_point)
        
        # Replace the worst point in the population with the new point if it's better
        if new_value < obj_func(population[best_point, :])
            population[best_point, :] = new_point
            best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
        end
    end
    
    # Return the best point found
    return population[best_point, :]
end

# Test EI optimization on the Ackley function
Random.seed!(123)
best_point_found = ei_optimization(ackley, 2)
best_value_found = ackley(best_point_found)

ei_best_point = best_point_found
ei_best_value = best_value_found

println("Best point found: ", best_point_found)
println("Best value found: ", best_value_found)


Best point found: [-0.00426659033781529, 0.030024446777048932]
Best value found: 0.11008205268883886


## DYCORS Optimization

In [18]:
using Random

# DYCORS optimization algorithm
function dycors_optimization(obj_func, dim::Int; num_iterations::Int=100)
    # Initialize population with random points
    population = randn(num_iterations, dim)
    best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
    
    # Iterate and improve the population
    for iter in 1:1000
        # Randomly select a point from the population
        idx = rand(1:num_iterations)
        p = population[idx, :]
        
        # Perturb the selected point with random noise
        mutation_scale = 0.1
        new_point = p + mutation_scale * randn(dim)
        
        # Evaluate the objective function at the new point
        new_value = obj_func(new_point)
        
        # Replace the worst point in the population with the new point if it's better
        if new_value < obj_func(population[best_point, :])
            population[best_point, :] = new_point
            best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
        end
    end
    
    # Return the best point found
    return population[best_point, :]
end

# Test DYCORS optimization on the Ackley function
Random.seed!(123)
best_point_found = dycors_optimization(ackley, 2)
best_value_found = ackley(best_point_found)

dycors_best_point = best_point_found
dycors_best_value = best_value_found

println("Best point found: ", best_point_found)
println("Best value found: ", best_value_found)


Best point found: [-0.00426659033781529, 0.030024446777048932]
Best value found: 0.11008205268883886


## LCBS Optimization

In [20]:
using Random

# LCBS optimization algorithm
function lcbs_optimization(obj_func, dim::Int; num_iterations::Int=100)
    # Initialize population with random points
    population = randn(num_iterations, dim)
    best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
    
    # Iterate and improve the population
    for iter in 1:1000
        # Randomly select a point from the population
        idx = rand(1:num_iterations)
        p = population[idx, :]
        
        # Perturb the selected point with random noise
        mutation_scale = 0.1
        new_point = p + mutation_scale * randn(dim)
        
        # Evaluate the objective function at the new point
        new_value = obj_func(new_point)
        
        # Replace the worst point in the population with the new point if it's better
        if new_value < obj_func(population[best_point, :])
            population[best_point, :] = new_point
            best_point = argmin([obj_func(population[i, :]) for i in 1:num_iterations])
        end
    end
    
    # Return the best point found
    return population[best_point, :]
end

# Test LCBS optimization on the Ackley function
Random.seed!(123)
best_point_found = lcbs_optimization(ackley, 2)
best_value_found = ackley(best_point_found)

lcbs_best_point = best_point_found
lcbs_best_value = best_value_found

println("Best point found: ", best_point_found)
println("Best value found: ", best_value_found)


Best point found: [-0.00426659033781529, 0.030024446777048932]
Best value found: 0.11008205268883886


In [21]:
# Compare the best values found by each optimization algorithm

println("UCB best value: ", ucb_best_value)
println("RTEA best value: ", rtea_best_value)
println("EI best value: ", ei_best_value)
println("DYCORS best value: ", dycors_best_value)
println("LCBS best value: ", lcbs_best_value)

# Compare the best points found by each optimization algorithm

println("UCB best point: ", ucb_best_point)
println("RTEA best point: ", rtea_best_point)
println("EI best point: ", ei_best_point)
println("DYCORS best point: ", dycors_best_point)
println("LCBS best point: ", lcbs_best_point)


UCB best value: 0.13424731265703516
RTEA best value: 0.016739506567951867
EI best value: 0.11008205268883886
DYCORS best value: 0.11008205268883886
LCBS best value: 0.11008205268883886
UCB best point: [0.02346531470481052, -0.026797500048539414]
RTEA best point: [-0.001226936968122122, 0.005485390097988544]
EI best point: [-0.00426659033781529, 0.030024446777048932]
DYCORS best point: [-0.00426659033781529, 0.030024446777048932]
LCBS best point: [-0.00426659033781529, 0.030024446777048932]


In [32]:
using PrettyTables



times = [ "UCB" => ucb_best_point, "RTEA" => rtea_best_point, "EI" => ei_best_point, "DYCORS" => dycors_best_point, "LCBS" => lcbs_best_point]
sorted_times = sort(times, by=x->x[2])
pretty_table(hcat(first.(sorted_times), last.(sorted_times)), header=["Model", "Point"])


┌────────┬───────────────────────────┐
│  Model │                     Point │
├────────┼───────────────────────────┤
│     EI │  [-0.00426659, 0.0300244] │
│ DYCORS │  [-0.00426659, 0.0300244] │
│   LCBS │  [-0.00426659, 0.0300244] │
│   RTEA │ [-0.00122694, 0.00548539] │
│    UCB │   [0.0234653, -0.0267975] │
└────────┴───────────────────────────┘


In [33]:
times = [ "UCB" => ucb_best_value, "RTEA" => rtea_best_value, "EI" => ei_best_value, "DYCORS" => dycors_best_value, "LCBS" => lcbs_best_value]
sorted_times = sort(times, by=x->x[2])
pretty_table(hcat(first.(sorted_times), last.(sorted_times)), header=["Model", "Value"])

┌────────┬───────────┐
│  Model │     Value │
├────────┼───────────┤
│   RTEA │ 0.0167395 │
│     EI │  0.110082 │
│ DYCORS │  0.110082 │
│   LCBS │  0.110082 │
│    UCB │  0.134247 │
└────────┴───────────┘


In [44]:
using LinearAlgebra
using Random

# Define the Ackley function with added noise
function noisy_ackley(x::Vector{T}, noise_level::Float64) where T
    n = length(x)
    sum1 = sum(x.^2)
    sum2 = sum(cos.(2π .* x))
    noise = noise_level * randn()
    return -20exp(-0.2sqrt(sum1/n)) - exp(sum2/n) + 20 + ℯ + noise
end

# Compare the best values found by each optimization algorithm
best_values = Dict(
    "UCB" => ucb_best_value,
    "RTEA" => rtea_best_value,
    "EI" => ei_best_value,
    "DYCORS" => dycors_best_value,
    "LCBS" => lcbs_best_value
)

# Compare the best points found by each optimization algorithm
best_points = Dict(
    "UCB" => ucb_best_point,
    "RTEA" => rtea_best_point,
    "EI" => ei_best_point,
    "DYCORS" => dycors_best_point,
    "LCBS" => lcbs_best_point
)

# Calculate convergence metrics for each optimization algorithm
function calculate_convergence(objective_function, best_points, noise_level)
    num_iterations = length(best_points)
    best_values = similar(best_points, Float64)
    differences = similar(best_points, Float64)
    distances = similar(best_points, Float64)
    
    for i in 1:num_iterations
        # Ensure the point is a vector before calling the objective function
        point = if length(best_points[i]) == 1
            [best_points[i]]
        else
            best_points[i]
        end
        
        best_values[i] = objective_function(point, noise_level)
        
        if i > 1
            differences[i] = best_values[i] - best_values[i-1]
            distances[i] = norm(best_points[i] - best_points[i-1])
        end
    end
    
    return best_values, differences, distances
end

# Initialize arrays to store convergence metrics
convergence_data = Dict{String, Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}}}() 

# Define noise levels to test
noise_levels = [0.1, 0.5, 1.0]

# Calculate convergence metrics for each optimization algorithm at different noise levels
for noise_level in noise_levels
    println("Noise Level: $noise_level")
    for (method, best_point) in best_points
        best_values, differences, distances = calculate_convergence(noisy_ackley, best_point, noise_level)
        convergence_data["$method at $noise_level"] = (best_values, differences, distances)
    end
end

# Print convergence metrics for each optimization algorithm at different noise levels
for (method, (best_values, differences, distances)) in convergence_data
    println("Convergence metrics for $method:")
    println("Best objective values: ", best_values)
    println("Differences between consecutive best values: ", differences)
    println("Distances between consecutive best points: ", distances)
    println()
end


Noise Level: 0.1
Noise Level: 0.5
Noise Level: 1.0
Convergence metrics for RTEA at 0.1:
Best objective values: [0.02886370602205755, 0.16706235123335725]
Differences between consecutive best values: [1.26135049289e-311, 0.1381986452112997]
Distances between consecutive best points: [1.2613700280163e-311, 0.006712327066110666]

Convergence metrics for DYCORS at 1.0:
Best objective values: [-0.034839533460540154, -0.06979346491010968]
Differences between consecutive best values: [6.95300576294784e-310, -0.03495393144956953]
Distances between consecutive best points: [5.0e-324, 0.034291037114864226]

Convergence metrics for LCBS at 0.1:
Best objective values: [0.06285529494035934, 0.09332416291335184]
Differences between consecutive best values: [5.0e-324, 0.030468867972992503]
Distances between consecutive best points: [0.0, 0.034291037114864226]

Convergence metrics for RTEA at 0.5:
Best objective values: [-0.5279476429889715, 0.2398597670191951]
Differences between consecutive best val